In [13]:
#2
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

gs


In [14]:
def display_menu_card(menu: str, db: Dict) -> str:
    return f"""
    ==============================
    🍵 {menu} 메뉴 카드
    ==============================
    • 설명: {info['description']}
    • 가격: {info['price']:,}원
    • 주요 재료: {', '.join(info['ingredients'])}
    ==============================
    """

In [16]:
import json
import requests
from typing import TypedDict, List, Dict
from langgraph.graph import StateGraph

# Qoob API 설정 (실제 키로 교체 필요)
QOOB_API_KEY = "your_qoob_api_key"
QOOB_API_URL = "https://api.qoob.ai/v1/chat/completions"

# 상태 정의
class CafeState(TypedDict):
    messages: List[Dict[str, str]]
    current_menu: str
    type: str
    menu_db: Dict[str, Dict]
    should_continue: bool

# 메뉴 DB 로드 함수
def load_menu_db(filepath: str) -> Dict[str, Dict]:
    with open(filepath, "r", encoding="utf-8") as f:
        return json.load(f)

# Qoob API 호출 함수
def call_qoob_api(prompt: str, context: str = "") -> str:
    headers = {
        "Authorization": f"Bearer {QOOB_API_KEY}",
        "Content-Type": "application/json"
    }
    
    data = {
        "model": "qoob-4",
        "messages": [
            {"role": "system", "content": "당신은 카페 메뉴 전문가입니다. 친절하고 전문적으로 답변해주세요."},
            {"role": "user", "content": f"{context}\n\n{prompt}"}
        ],
        "temperature": 0.7
    }
    
    try:
        response = requests.post(QOOB_API_URL, headers=headers, json=data, timeout=10)
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]
    except Exception as e:
        return f"⚠️ 오류 발생: {str(e)}"

# 노드 정의 --------------------------------------------------

def init_state(state: CafeState) -> dict:
    if not state:
        state = {
            "messages": [{"role": "system", "content": "카페 메뉴 안내 시스템을 시작합니다."}],
            "current_menu": "",
            "type": "",
            "menu_db": load_menu_db("C:/mylangchain/langchain_basic/data/cafe_menu_data.txt"),
            "should_continue": True
        }
    return state

def display_menu_card(menu: str, db: Dict) -> str:
    if menu not in db:
        return f"⚠️ {menu} 메뉴를 찾을 수 없습니다."
    
    info = db[menu]
    return f"""
    ==============================
    🍵 {menu} 메뉴 카드
    ==============================
    • 설명: {info['description']}
    • 가격: {info['price']:,}원
    • 주요 재료: {', '.join(info['ingredients'])}
    ==============================
    """

def booq_greeting(state: CafeState) -> dict:
    greeting = call_qoob_api(
        "친절하게 인사하고 사용자에게 어떤 메뉴를 도와줄지 물어봐줘.",
        "현재 제공 메뉴: " + ", ".join(state["menu_db"].keys())
    )
    state["messages"].append({"role": "assistant", "content": greeting})
    print(f"\n🤖 booq: {greeting}")
    return state

def get_user_input(state: CafeState) -> dict:
    user_input = input("\n👤 사용자: ").strip()
    
    if user_input.lower() in ["종료", "exit", "quit", "y"]:
        state["should_continue"] = False
        return state
    
    if not user_input:
        state["messages"].append({"role": "user", "content": "help"})
        print("🤖 booq: 메뉴 이름을 입력해주세요. 예) '카푸치노 정보 알려줘'")
        return state
    
    # 더 정확한 메뉴 인식 로직
    menu = ""
    for word in user_input.split():
        normalized_word = word.lower().replace(" ", "")
        for menu_name in state["menu_db"].keys():
            if normalized_word in menu_name.lower().replace(" ", ""):
                menu = menu_name
                break
        if menu:
            break
    
    state["current_menu"] = menu if menu else ""
    state["messages"].append({"role": "user", "content": user_input})
    return state

def check_continue(state: CafeState) -> str:
    return "end" if not state["should_continue"] else "continue"

def classify_request(state: CafeState) -> dict:
    last_msg = state["messages"][-1]["content"].lower()
    
    if any(word in last_msg for word in ["추천", "추천해줘", "뭐 먹지"]):
        state["type"] = "recommend"
    elif any(word in last_msg for word in ["가격", "얼마", "비용"]):
        state["type"] = "price"
    elif any(word in last_msg for word in ["재료", "성분", "구성"]):
        state["type"] = "ingredients"
    else:
        state["type"] = "info"
    return state

def provide_info(state: CafeState) -> dict:
    menu = state["current_menu"]
    db = state["menu_db"]
    
    if menu:
        menu_card = display_menu_card(menu, db)
        print(menu_card)
        
        additional_info = call_qoob_api(
            f"다음 메뉴에 대해 친절하게 설명해주세요: {menu}",
            f"메뉴 정보: {db.get(menu, {})}"
        )
        print(f"\n🤖 booq: {additional_info}\n")
        
        state["messages"].extend([
            {"role": "assistant", "content": menu_card},
            {"role": "assistant", "content": additional_info}
        ])
    else:
        response = "🤖 booq: 어떤 메뉴가 궁금하신가요? 메뉴 이름을 정확히 입력해주세요."
        print(response)
        state["messages"].append({"role": "assistant", "content": response})
    
    return state

def recommend_menu(state: CafeState) -> dict:
    db = state["menu_db"]
    prompt = "현재 메뉴: " + ", ".join(db.keys()) + "\n다양한 취향을 고려해 3가지 메뉴를 추천해주고, 각각 간단히 설명해줘."
    recommendation = call_qoob_api(prompt)
    print(f"\n🤖 booq: {recommendation}\n")
    state["messages"].append({"role": "assistant", "content": recommendation})
    return state

def end_conversation(state: CafeState) -> dict:
    farewell = call_qoob_api("작별 인사를 친절하게 해줘.")
    print(f"\n🤖 booq: {farewell}")
    state["messages"].append({"role": "assistant", "content": farewell})
    return state

# 그래프 구성 --------------------------------------------------
workflow = StateGraph(CafeState)

# 노드 추가
workflow.add_node("init", init_state)
workflow.add_node("greet", booq_greeting)
workflow.add_node("get_input", get_user_input)
workflow.add_node("check_continue", check_continue)
workflow.add_node("classify", classify_request)
workflow.add_node("info", provide_info)
workflow.add_node("recommend", recommend_menu)
workflow.add_node("end", end_conversation)

# 흐름 구성
workflow.set_entry_point("init")
workflow.add_edge("init", "greet")
workflow.add_edge("greet", "get_input")
workflow.add_edge("get_input", "check_continue")

# 계속 여부 확인 분기
workflow.add_conditional_edges(
    "check_continue",
    lambda state: "end" if not state["should_continue"] else "classify",
)

# 요청 분류
workflow.add_conditional_edges(
    "classify",
    lambda state: state["type"],
    {
        "info": "info",
        "price": "info",
        "recommend": "recommend",
        "ingredients": "info"
    }
)

# 정보 제공 후 다시 시작
workflow.add_edge("info", "greet")
workflow.add_edge("recommend", "greet")
workflow.set_finish_point("end")

# 컴파일 (재귀 제한 문제 해결을 위해 그래프 구조 최적화)
cafe_bot = workflow.compile()

# 실행 --------------------------------------------------
if __name__ == "__main__":
    print("\n" + "="*50)
    print("프리미엄 카페 메뉴 안내 시스템 (Qoob AI 활용)")
    print("="*50)
    
    # 초기 상태
    state = {}
    
    # 실행 예시
    print("\n실행 예시:")
    print("👤 사용자: 카푸치노 정보 알려줘")
    print(display_menu_card("카푸치노", load_menu_db("C:/mylangchain/langchain_basic/data/cafe_menu_data.txt")))
    print("👤 사용자: 종료")
    print("🤖 booq: 감사합니다! 또 방문해주세요 :)")
    print("="*50)
    
    # 시스템 실행 (재귀 문제 방지를 위해 최대 10회 반복으로 제한)
    max_iterations = 10
    current_iteration = 0
    
    try:
        state = cafe_bot.invoke(state)
        while state.get("should_continue", True) and current_iteration < max_iterations:
            state = cafe_bot.invoke(state)
            current_iteration += 1
            
        if current_iteration >= max_iterations:
            print("\n🤖 booq: 최대 대화 횟수에 도달했습니다. 새로운 대화를 시작해주세요.")
    except Exception as e:
        print(f"\n⚠️ 시스템 오류: {str(e)}")
        print("🤖 booq: 문제가 발생했습니다. 다시 시도해주세요.")


프리미엄 카페 메뉴 안내 시스템 (Qoob AI 활용)

실행 예시:
👤 사용자: 카푸치노 정보 알려줘

    🍵 카푸치노 메뉴 카드
    • 설명: 에스프레소, 스팀 밀크, 우유 거품이 1:1:1 비율로 구성된 이탈리아 전통 커피입니다. 진한 커피 맛과 부드러운 우유 거품의 조화가 일품이며, 계피 파우더를 뿌려 제공합니다.
    • 가격: 5,000원
    • 주요 재료: 에스프레소, 스팀 밀크, 우유 거품
    
👤 사용자: 종료
🤖 booq: 감사합니다! 또 방문해주세요 :)

🤖 booq: ⚠️ 오류 발생: HTTPSConnectionPool(host='api.qoob.ai', port=443): Max retries exceeded with url: /v1/chat/completions (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000013F33DD9590>: Failed to resolve 'api.qoob.ai' ([Errno 11001] getaddrinfo failed)"))

⚠️ 시스템 오류: Expected dict, got continue
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_GRAPH_NODE_RETURN_VALUE
🤖 booq: 문제가 발생했습니다. 다시 시도해주세요.
